<a href="https://colab.research.google.com/github/esteban-miller/3.Transformacion-modelado-de-datos/blob/main/Modulo_3_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Intergrantes:
#
# Sergio Martin
# Esteban Miller
# Jaquelin Da Costa
# Roberto De Gouveia

!pip install unidecode

import pandas as pd
from unidecode import unidecode

# Cargar datos
archivos = ['barcelona.xlsx', 'valencia.xlsx', 'pozuelo.xlsx', 'mallorca.xlsx', 'malasaña.xlsx']

tabla_total = pd.concat([pd.read_excel(archivo) for archivo in archivos])

#df_barcelona = pd.read_excel("C:/Users/jaque/Documents/IMMUNE/Modulo3/Ejercicio Final/barcelona.xlsx")
#df_malasana = pd.read_excel("C:/Users/jaque/Documents/IMMUNE/Modulo3/Ejercicio Final/malasaña.xlsx")
#df_mallorca = pd.read_excel("C:/Users/jaque/Documents/IMMUNE/Modulo3/Ejercicio Final/mallorca.xlsx")
#df_pozuelo = pd.read_excel("C:/Users/jaque/Documents/IMMUNE/Modulo3/Ejercicio Final/pozuelo.xlsx")
#df_valencia = pd.read_excel("C:/Users/jaque/Documents/IMMUNE/Modulo3/Ejercicio Final/valencia.xlsx")
#
#tabla_total = pd.concat([df_barcelona,df_malasana,df_mallorca,df_pozuelo,df_valencia], ignore_index=True)
#tabla_total = pd.read_excel("C:/Users/jaque/Documents/IMMUNE/Modulo3/Ejercicio Final/barcelona.xlsx")

#Información general de la tabla
tabla_total.dtypes
tabla_total.size
tabla_total.columns
tabla_total.info()
tabla_total.isnull().sum()
x = tabla_total.describe()

# Convertimos todo a minuscula y limpiamos antes y despues
tabla_total.columns= tabla_total.columns.str.lower().str.strip()

# Quitamos todos los caracteres diacriticos
tabla_total.columns = [unidecode(col) for col in tabla_total.columns]

#Nos percatamos de que hay columnas que tienen todos los datos nulos.

# =============================================================================
# VALORES NULOS
# =============================================================================

# Iterar y eliminar filas con número y fecha nulos
for index, row in tabla_total.iterrows():
    if pd.isnull(row['numero']) and pd.isnull(row['fecha']):
        tabla_total.drop(index, inplace=True)

# Cuantos valores nulos hay de cada variable
tabla_total['familia'].fillna("NO INFO", inplace = True)

#Quitamos el check de "Local"
tabla_total.local = tabla_total.local.str[:-4]

#Tabla copia para hacer pruebas en el dataset original
tabla_copia = tabla_total.copy()

# Eliminamos las columnas que tienen todos los valores nulos,
# cod. promocion, cod. descuento, grupo mayor
tabla_total.isnull().sum()
tabla_total.columns[tabla_total.isnull().any()]

tabla_total.drop(["cod. promocion", "grupo mayor", "cif", "cuenta contable", "ciudad", "provincia", "calle", "codigo postal"], axis=1, inplace=True)

# =============================================================================
# Proyecto Final / Proyecto de Análisis de Ventas
# =============================================================================


# =============================================================================
# Preguntas de Análisis:
# =============================================================================


# =============================================================================
# 1. Análisis Temporal:
# =============================================================================
# =============================================================================
# ¿Cuál es el patrón temporal de las ventas en Barcelona, Valencia, Pozuelo,
# Mallorca y Malasaña a lo largo de los días y año?
# =============================================================================

# Asegurar que la columna 'fecha' esté en formato datetime
tabla_total['fecha'] = pd.to_datetime(tabla_total['fecha'], errors='coerce')

# Extraer características temporales
tabla_total['year'] = tabla_total['fecha'].dt.year
tabla_total['month'] = tabla_total['fecha'].dt.month
tabla_total['day_of_week'] = tabla_total['fecha'].dt.day_name()

# Resumen de ventas por día
resumen_diario = tabla_total.groupby([tabla_total['fecha'].dt.date, 'local'])['total'].sum().reset_index()

# Resumen mensual
resumen_mensual = tabla_total.groupby(['year', 'month','local'])['total'].sum().reset_index()



# =============================================================================
# ¿Existen variaciones estacionales o eventos especiales que impacten
# significativamente las ventas en cada ubicación?
# =============================================================================

#Realizamos una segmentacion por estaciones dividiendo por le numero del mes

resumen_mensual["estaciones"] =pd.cut(resumen_mensual.month,bins=[0,3,6,8,12],
                               labels=["invierno","primavera","verano","otoño"])

#Agrupamos y sumamos todos los totales por mes, año y local
resumen_anual = resumen_mensual.groupby(['year','estaciones','local'])['total'].sum().reset_index()

# =============================================================================
# 2. Análisis Geográfico:
# =============================================================================
# =============================================================================
# ¿Cómo se comparan las ventas totales y promedios entre las cinco
# ubicaciones?
# =============================================================================

# Agrupé los datos por 'ubicacion' y calculé las ventas totales y el promedio
resumen_ventas = tabla_total.groupby('ubicacion').agg(
    ventas_totales=('total', 'sum'),
    ventas_promedio=('total', 'mean')
).reset_index()

# =============================================================================
# ¿Cuál es la distribución de clientes y cuáles son los productos más
# populares en cada ubicación?
# =============================================================================

# Contar el número de clientes únicos en cada ubicación
clientes_por_ubicacion = tabla_total.groupby('ubicacion')['cliente'].nunique().reset_index().rename(columns={'cliente': 'num_clientes'})

# Busqué los productos más populares en cada ubicación
productos_populares = tabla_total.groupby(['ubicacion', 'producto']).size().reset_index(name='frecuencia')
productos_populares = productos_populares.sort_values(by=['ubicacion', 'frecuencia'], ascending=[True, False])

# Obtuve el producto más popular en cada ubicación
productos_mas_populares = productos_populares.loc[productos_populares.groupby('ubicacion')['frecuencia'].idxmax()]

# =============================================================================
# 3. Análisis de Descuentos:
# =============================================================================
# =============================================================================
# ¿Cómo afectan los descuentos al aumento de las ventas en cada ubicación?
# =============================================================================

# Calculo del total de ventas sin descuento y con descuento
tabla_total['ventas_sin_dto'] = tabla_total['cantidad'] * tabla_total['precio']
tabla_total['ventas_con_dto'] = tabla_total['ventas_sin_dto'] - tabla_total['dto. eur']

# Ubicación y calculo de las ventas totales sin descuento y con descuento agrupadas
ventas_por_ubicacion = tabla_total.groupby('ubicacion').agg(
    ventas_totales_sin_dto=('ventas_sin_dto', 'sum'),
    ventas_totales_con_dto=('ventas_con_dto', 'sum')
).reset_index()

# Impacto del descuento en las ventas
ventas_por_ubicacion['impacto_descuento'] = ventas_por_ubicacion['ventas_totales_con_dto'] / ventas_por_ubicacion['ventas_totales_sin_dto'] - 1

# =============================================================================
# ¿Cuáles son los tipos de descuentos más utilizados y cuál es su impacto en
#  los ingresos?
# =============================================================================

# Identificar los tipos de descuentos más utilizados
tipos_descuentos_utilizados = tabla_total.groupby('cod. descuento').size().reset_index(name='frecuencia').sort_values(by='frecuencia', ascending=False)

impacto_descuentos = tabla_total.groupby('cod. descuento').agg(
    ventas_totales_sin_dto=('ventas_sin_dto', 'sum'),
    ventas_totales_con_dto=('ventas_con_dto', 'sum')
).reset_index()

impacto_descuentos['impacto'] = impacto_descuentos['ventas_totales_con_dto'] / impacto_descuentos['ventas_totales_sin_dto'] - 1

# =============================================================================
# 4. Comparación de Precios:
# =============================================================================
# =============================================================================
# ¿Existe variación significativa en los precios de productos similares entre
#  las ubicaciones?
# =============================================================================
ubicaciones = tabla_total['ubicacion'].unique()

# Opcion 1

# Calcular la media de precios por producto y ubicación
tabla_media = tabla_total.pivot_table(index='producto', columns='ubicacion', values='precio', aggfunc='mean')

# Restablecer el índice para que 'producto' sea una columna normal
tabla_media.reset_index(inplace=True)

# Opcion 2

ventas_ubicaciones = tabla_total.groupby(['ubicacion','producto']).agg(
    media_precio=('precio', 'mean'),
).reset_index()

# =============================================================================
# ¿Qué diferencias estratégicas en precios pueden identificarse entre
#  Barcelona, Valencia, Pozuelo, Mallorca y Malasaña?
# =============================================================================

ubicaciones = tabla_total['local'].unique()


# Calcular la media de precios por producto y ubicación
tabla_media = tabla_total.pivot_table(index='producto', columns='local', values='precio', aggfunc='mean')

# Restablecer el índice para que 'producto' sea una columna normal
tabla_media.reset_index(inplace=True)

# =============================================================================
# 5. Productos Más Vendidos:
# =============================================================================
# =============================================================================
# ¿Cuáles son los productos más vendidos en cada ubicación?
# =============================================================================

top_ventas_ubicacion = tabla_total.groupby(['local','producto'])['cantidad'].sum().reset_index(name="vendidos").sort_values(by='vendidos', ascending=False)

# He creado una función para obtener los x primeros productos más vendidos
# en cada ubicación. Creo que esto agilizara la consutla para el estudio de
# las ventas.

def top_x_productos(x):

    top_por_ubicacion = (
        top_ventas_ubicacion.groupby('local')
        .head(x)  # Seleccionar los primeros 5 por grupo
        .reset_index(drop=True)
        .sort_values(by='local', ascending=True)
    )

    return top_por_ubicacion

top_1_por_ubicacion = top_x_productos(1)
top_5_por_ubicacion = top_x_productos(5)


# =============================================================================
# ¿Cómo varía la popularidad de los productos a lo largo de diferentes
# meses y días en cada ubicación?
# =============================================================================

tabla_total['mes'] = tabla_total['fecha'].dt.month
tabla_total['dia'] = tabla_total['fecha'].dt.day


ventas_por_mes_dia = tabla_total.groupby(['local', 'producto', 'mes', 'dia'])['cantidad'].sum().reset_index(name='vendidos')

promedio_mensual = ventas_por_mes_dia.groupby(['local', 'producto', 'mes'])['vendidos'].mean().reset_index(name='promedio_mensual')

promedio_diario =  ventas_por_mes_dia.groupby(['local', 'producto', 'dia'])['vendidos'].mean().reset_index(name='promedio_diario')

variacion_mensual_diaria = (
    ventas_por_mes_dia
    .merge(promedio_mensual, on=['local', 'producto', 'mes'], how='left')
    .merge(promedio_diario, on=['local', 'producto', 'dia'], how='left')
)

variacion_mensual_diaria['variacion_mensual'] = (
    (variacion_mensual_diaria['vendidos'] - variacion_mensual_diaria['promedio_mensual']) /
    variacion_mensual_diaria['promedio_mensual']
) * 100

variacion_mensual_diaria['variacion_diaria'] = (
    (variacion_mensual_diaria['vendidos'] - variacion_mensual_diaria['promedio_diario']) /
    variacion_mensual_diaria['promedio_diario']
) * 100


# Ordenar por mayor variación mensual
variacion_mensual_top = variacion_mensual_diaria.sort_values(by='variacion_mensual', ascending=False)

# Ordenar por mayor variación diaria
variacion_diaria_top = variacion_mensual_diaria.sort_values(by='variacion_diaria', ascending=False)



# =============================================================================
# 6. Clientes Comunes:
# =============================================================================
# =============================================================================
# ¿Existen clientes que han realizado compras en más de una ubicación?
# =============================================================================
clientes_local = tabla_total.groupby('cliente')['ubicacion'].nunique().reset_index(name='num_ubicaciones')

clientes_m1u = clientes_local.loc[clientes_local["num_ubicaciones"] > 1]

# =============================================================================
# ¿Cómo contribuyen estos clientes a las ventas totales y cuál es su
# comportamiento de compra en diferentes ubicaciones?
# =============================================================================

lista_clientes_m1u = clientes_m1u['cliente'].tolist()

ventas_clientes_m1u = tabla_total[tabla_total["cliente"].isin(lista_clientes_m1u)]

suma_ventas_m1u = ventas_clientes_m1u.groupby('cliente')['precio'].sum().reset_index(name='suma_ventas_m1u')

suma_ventas_m1u["total_ventas"] = tabla_total["precio"].sum()

suma_ventas_m1u["porcentaje_ventas_m1u"] = ((suma_ventas_m1u["suma_ventas_m1u"]/suma_ventas_m1u["total_ventas"]) * 100).round(2)

# =============================================================================
# 7. Análisis Detallado del Comportamiento de Ventas:
# =============================================================================
# =============================================================================
# ¿Cuál es la duración promedio de una transacción? (Calcula la diferencia
#  entre la fecha de creación y la fecha de venta).
# =============================================================================

#primero verificamos que tanto "fecha" como "fecha de creacion"
#se encuentren en tipo datetime, si es asi realizamos lo siguiente:

# Calcular la diferencia entre'fecha creacion' y 'fecha' en horas

tabla_total['duracion_transaccion'] = (tabla_total['fecha creacion'] - tabla_total['fecha']).dt.total_seconds() / (60 * 60)  # en horas

# =============================================================================
# ¿En qué día de la semana se realizan más ventas en promedio?
# =============================================================================

# primero asigno cada día de la semana a la columna 'fecha'con dayofweek,
# lunes = 0 , martes = 1 etc..

tabla_total['dia_semana'] = tabla_total['fecha'].dt.dayofweek

# contamos el total de ventas por dia
ventas_por_dia = tabla_total.groupby('dia_semana').size()

# vemos cual es el día de la semana con más ventas en promedio
dia_con_mas_ventas = ventas_por_dia.idxmax()

## hasta ahi ya tengo lo que me piden pero nose si el quiere que se vea reflejado
## en la tabla o asi estaria bien

# =============================================================================
#¿Cuáles son los meses con las ventas más altas y más bajas?
# =============================================================================

# Extraer el mes de la columna 'fecha'
tabla_total['mes'] = tabla_total['fecha'].dt.month

# Sumar las ventas por mes.
ventas_por_mes = tabla_total.groupby('mes')['total'].sum()

# Identificar el mes con las ventas más altas y más bajas
mes_ventas_max = ventas_por_mes.idxmax()
mes_ventas_min = ventas_por_mes.idxmin()

# =============================================================================
# ¿Cuál es el porcentaje de descuento promedio en relación con el precio
#  original de los productos?
# =============================================================================

# Calcular el porcentaje de descuento promedio en comparacion al precio original
descuento_promedio = tabla_total['dto. %'].mean()

# resultado
print(f"El porcentaje de descuento promedio es: {descuento_promedio:.2f}%")

# =============================================================================
# 8. Preguntas adicionales:
# =============================================================================
# =============================================================================
# 8.1 Crear una columna que indique si una venta tuvo un descuento y que
# porcentaje de ventas tuvieron descuento
# =============================================================================

# Crear la nueva columna 'tiene_descuento'
tabla_total['tiene_descuento'] = (tabla_total['dto. %'] > 0) | (tabla_total['dto. eur'] > 0)

# Calcular el porcentaje de ventas con descuento
ventas_con_descuento = tabla_total['tiene_descuento'].sum()
len_total = len(tabla_total)
porcentaje_con_descuento = (ventas_con_descuento / len_total) * 100


# =============================================================================
# 8.2. Crear una columna de ganancia neta restando el costo de los
# productos al total
# =============================================================================

tabla_total['ganancia_neta'] = tabla_total['total'] - tabla_total['precio coste']

# =============================================================================
# 8.3 Crear una columna que clasifique las ventas como "Alta", "Media" o
# "Baja" según el monto total
# =============================================================================
#Opcion 1
tabla_total["clasificacion_ventas"] = pd.cut(tabla_total.total,bins=[tabla_total.total.min(),0,tabla_total.total.mean(),tabla_total.total.max()],
                               labels=["Baja","Media","Alta"])

#Opcion 2
tabla_total["clasificacion_ventas"] = pd.cut(tabla_total.total,bins=[tabla_total.total.min(),tabla_total.total.mean(),450,tabla_total.total.max()],
                               labels=["Baja","Media","Alta"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.3 MB/s eta 0:00:00


FileNotFoundError: [Errno 2] No such file or directory: 'barcelona.xlsx'



# Documentación y Presentación del Informe Final:



---



Limpieza de datos

Tras realizar un análisis preliminar del tipo de variables del dataset con métodos como dtypes, .size, .columns o .info() entre otros, comprobamos cuántas de estas variables tenían datos que debían ser preprocesados para facilitar su posterior análisis. Mostramos la cantidad de datos nulos y los cuantificamos para tomar decisiones sobre qué variables eliminar y cuáles conservar.
Posteriormente, realizamos las primeras transformaciones de los datos, limpiando los nombres de las variables con .strip() y poniéndolos todos en minúsculas con .lower. También realizamos un unidecode para eliminar todo tipo de caracteres especiales.
En cuanto a la limpieza de nulos, inicialmente iteramos sobre la variable fecha para eliminar las filas con fechas nulas mediante un bucle for. Imputamos el valor "NO INFO" para los valores nulos de la variable familia y limpiamos la variable local usando índices. Eliminamos las variables con un alto contenido de valores nulos que no eran relevantes para el análisis, como cod_promoción, cod_descuento y grupo_mayor. Por precaución, creamos un nuevo dataframe con todas esas variables eliminadas, para facilitar el acceso a las mismas en el futuro.

---
1.	Análisis Temporal

Limpiamos la variable fecha con el método pd.to_datetime(tabla_total['fecha'], errors='coerce'), asegurándonos de que todos los valores tienen el formato adecuado. Posteriormente, creamos nuevas columnas con los valores de año, mes y día para acceder con mayor facilidad a los datos de fechas.
Para obtener el resumen de ventas por día, realizamos un groupby por fecha y local, sumando su total. Hicimos lo mismo a nivel mensual, agrupando esta vez por mes.
Con el objetivo de identificar las variaciones estacionales, utilizamos el método pd.cut para segmentar la variable fecha en las cuatro estaciones. Esto nos permitió observar las variaciones a lo largo de las estaciones, agrupando por año y estación.

---
2. Análisis Geográfico

Agrupamos por ubicación y calculamos la suma y la media del total de ventas.
Agrupamos también por el número de clientes únicos por ubicación y lo guardamos en una columna.
Filtramos por los productos más populares por ubicación usando .size(), lo cual nos da el número de veces que cada producto aparece en cada ubicación.
Ordenamos esta información con .sort_values().
Finalmente, usamos .idxmax() para obtener la fila completa del producto y ubicación con mayor frecuencia.

---
3. Análisis de Descuentos

Primero realizamos una operación para obtener las ventas con y sin descuento, para posteriormente hacer un .groupby() por ubicación y aplicar una función de agregación que nos permita obtener el total sumado de ambos tipos de ventas en cada lugar.

Finalmente, calculamos el porcentaje de ventas totales comparándolo con las ventas totales con descuento.

Para identificar los tipos de descuentos más utilizados, aplicamos .size() para contar cuántas filas tuvieron cada descuento y cuantificarlos en una nueva columna llamada "frecuencia", ordenada con .sort_values().

Después agrupamos por "cod. descuento" y usamos .agg() para sumar las ventas con y sin descuento, permitiéndonos calcular el porcentaje de impacto en las ventas totales dividiendo el total con descuento entre el total sin descuento.

---
4. Comparación de Precios

Obtenemos información sobre los valores únicos en la variable ubicación y, posteriormente, usamos el método pivot_table, que nos permite visualizar la información de un vistazo. Filtramos por la columna ubicación y calculamos la media de la columna precio. Podemos realizar un reset_index para recuperar los índices de la columna, aunque en este caso no es necesario.
Para realizar el mismo filtrado por local, simplemente modificamos el argumento y obtenemos el resultado: un dataframe con todas las ubicaciones una tras otra.
Aunque podríamos realizar la consulta de otro modo usando un groupby, la información se mostraría de una manera mucho menos visual (una ubicación debajo de otra), dificultando mucho más la comparación entre ellas.

---
5. Productos Más Vendidos


Aquí tienes tu texto corregido, Bro, manteniendo la estructura pero con mejor claridad y gramática:

Para saber cuáles son los productos más vendidos por ubicación, agrupamos por "local" y "producto", sumamos la cantidad y ordenamos los valores. Luego, guardamos esta información en una nueva columna llamada "vendidos".

Con una función sencilla, agrupamos por "local", seleccionamos el número del Top X de productos que queremos obtener y los ordenamos por local. Usamos el método .head() para extraer los productos más vendidos en cada ubicación dentro de la variable "local".

Usando datetime, agrupamos los datos por mes y día, aplicando .sum() para obtener la cantidad total de productos vendidos en cada periodo. Posteriormente, calculamos la media mensual y diaria con .mean(), utilizando la columna "vendidos" que creamos anteriormente.

Con los datasets resultantes (promedio_mensual y promedio_diario), realizamos un .merge() para unirlos y poder operar con los datos combinados. En este caso, necesitamos calcular la variación mensual y diaria de las ventas, por lo que creamos nuevas columnas donde restamos el total vendido al promedio mensual y lo dividimos entre este mismo. Multiplicándolo por 100, obtenemos el porcentaje de aumento o descenso con respecto al promedio mensual o diario.

Finalmente, ordenamos los resultados para visualizar con mayor claridad dónde hubo mayores diferencias.

---
6. Clientes Comunes

Con un .nunique y un .groupby podemos saber cuántos clientes únicos hay en las diferentes ubicaciones. Con esta información, convertimos en lista el dataset usando .to_list, filtramos nuestro dataset original por las ubicaciones donde están estos clientes únicos que han realizado compras en más de una ubicación, y con .isin los seleccionamos.
Sumamos el precio por cliente y lo comparamos con el sumatorio total de precios para, de este modo, obtener el porcentaje total de gasto que aportaron al contante final.

---

7. Análisis Detallado del Comportamiento de Ventas:

7.1

tabla_copia['duracion_transaccion'] = (tabla_copia['fecha creacion'] - tabla_copia['fecha']).dt.total_seconds() / (60 * 60)

1.	**`tabla_copia['fecha creacion'] - tabla_copia['fecha']`**:      Aquí se están restando dos columnas del DataFrame `tabla_copia`: la fecha de creación (`fecha creacion`) y la fecha de venta (`fecha`). El resultado de esta resta es un **`timedelta`** que contiene la diferencia de tiempo entre las dos fechas.

2.	**`.dt.total_seconds()`**:   
   Luego, se usa `.dt.total_seconds()` para convertir esa diferencia en **segundos**. Esto convierte el `timedelta` en un valor numérico que nos da la cantidad total de segundos entre las dos fechas.

7.2--------------------------------------------------------------------------

1.	**`tabla_copia['fecha'].dt.dayofweek`**:   
   Aquí estamos usando `.dt.dayofweek` para obtener el número que representa el **día de la semana** de la columna `fecha`. El valor devuelto es un entero donde:    - **0** representa lunes    - **1** representa martes    - **6** representa domingo.


2.	**`groupby('dia_semana')`**:   
   Agrupamos los datos por la columna `dia_semana`, que contiene los días de la semana. Luego, usamos `.size()` para contar cuántas ventas hubo en cada día de la semana.


3.	**`idxmax()`**:   
   Este método nos devuelve el **índice** (en este caso, el día de la semana) del valor máximo en la serie `ventas_por_dia`, es decir, el día de la semana en el que se realizaron más ventas.

### **Identificación de los meses con más y menos ventas**

```python
tabla_copia['mes'] = tabla_copia['fecha'].dt.month
```

1.	**`tabla_copia['fecha'].dt.month`**:   
   Con `.dt.month`, extraemos el **mes** de la columna `fecha`, que nos da un valor entero entre 1 y 12, representando los meses de enero a diciembre.

```python
ventas_por_mes = tabla_copia.groupby('mes')['total'].sum()
```

2.	**`groupby('mes')`**:   
   Agrupamos los datos por mes. Luego, usamos `.sum()` para sumar el valor de las ventas (`total`) de cada mes. Esto nos da el total de ventas de cada mes.

```python
mes_ventas_max = ventas_por_mes.idxmax()
mes_ventas_min = ventas_por_mes.idxmin()
```

3.	**`idxmax()` y `idxmin()`**:   
-	`idxmax()` nos da el **mes con las ventas más altas**.
-	`idxmin()` nos da el **mes con las ventas más bajas**.

 7.3--------------------------------------------------------------------------

1.	**`tabla_copia['dto. %']`**:   
   Accedemos a la columna que contiene el porcentaje de descuento en cada transacción (presumiblemente llamada `dto. %`).

2.	**`.mean()`**:   
  Usamos `.mean()` para calcular el **promedio** de los porcentajes de descuento.

3.	**`print()`**:   
   Finalmente, imprimimos el resultado del cálculo, mostrando el porcentaje de descuento promedio con dos decimales.

---
8. Preguntas adicionales:

Para saber si una venta tuvo o no un descuento, es sencillo realizar una consulta con dos condiciones en las variables que representan los descuentos “dto. %” y “dto. eur”. En el caso de que una de estas sea mayor que 0, se añadirá a la variable.

Para obtener el porcentaje total de ventas que tuvieron descuento, realizamos la suma de todas estas ventas con descuento y hacemos una operación para sacar el porcentaje con el total de registros de ventas del DataFrame, sirviéndonos de la función .len() que nos da esta información.

Para obtener el dato de la ganancia neta, únicamente seleccionamos la variable del “total” y la de “precio de coste” y las restamos.

Y finalmente, para realizar una segmentación en función de los datos de ventas, usamos la variable “total” para realizar un pd.cut() y segmentar, en este caso, en función del mínimo, la media y el valor máximo de esta columna, asignando la clasificación "Alta", "Media" o "Baja" con respecto a los datos totales de esta variable.
